In [1]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
### Local Tracing 

import os 
from agents import Agent, Runner, trace, set_trace_processors
from agents.tracing.processor_interface import TracingProcessor

class LocalTracingProcessor(TracingProcessor):
    def __init__(self):
        self.traces = []
        self.spans = []

    def on_trace_start(self, trace):
        self.traces.append(trace)
        print(f"Trace started: {trace}")
        # print(f"Trace ID: {trace.id}")

    def on_span_start(self, span):
        self.spans.append(span)
        print(f"Span started: {span}")
        # print(f"Span ID: {span.id}")
        # print(f"Parent Span ID: {span.parent_id}")
        print(span.export())

    def on_span_end(self, span):
        print(f"Span ended: {span}")
        # print(f"Span ID: {span.id}")
        # print(f"Parent Span ID: {span.parent_id}")
        print(span.export())

    def on_trace_end(self, trace):
        print(f"Trace ended: {trace}")
        # print(f"Trace ID: {trace.id}")
        print(trace.export())

    def force_flush(self):
        print("Flushing traces and spans...")

    def shutdown(self):
        print("Shutting down tracing processor...")
        print(f"Total traces: {len(self.traces)}")
        print(f"Total spans: {len(self.spans)}")
        for trace in self.traces:
            print(trace.export())
        for span in self.spans:
            print(span.export())

##setup the trace processcr
tracing_processor = LocalTracingProcessor()
set_trace_processors([tracing_processor])

async def main():
    
    agent = Agent(name = "Tracing Agent", instructions = "This agent will trace its execution.")

    with trace("Lifecyvle Trace"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function
import asyncio
asyncio.run(main())

Trace started: <agents.tracing.traces.TraceImpl object at 0x000001F9EED98590>
Span started: <agents.tracing.spans.SpanImpl object at 0x000001F9EEC119D0>
{'object': 'trace.span', 'id': 'span_a57e572bf10145f28a03004a', 'trace_id': 'trace_02709a2bdaa547218b32be3bf73c7dc8', 'parent_id': None, 'started_at': '2025-07-07T07:26:37.931528+00:00', 'ended_at': None, 'span_data': {'type': 'agent', 'name': 'Tracing Agent', 'handoffs': [], 'tools': None, 'output_type': 'str'}, 'error': None}
Span started: <agents.tracing.spans.SpanImpl object at 0x000001F9EEC11FD0>
{'object': 'trace.span', 'id': 'span_d4bc268848874d43842d3ce2', 'trace_id': 'trace_02709a2bdaa547218b32be3bf73c7dc8', 'parent_id': 'span_a57e572bf10145f28a03004a', 'started_at': '2025-07-07T07:26:38.548372+00:00', 'ended_at': None, 'span_data': {'type': 'response', 'response_id': None}, 'error': None}
Span ended: <agents.tracing.spans.SpanImpl object at 0x000001F9EEC11FD0>
{'object': 'trace.span', 'id': 'span_d4bc268848874d43842d3ce2', 't